#### Import Libraries

In [1]:
!pip install accelerate -U
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.0
    Uninstalling huggingface-hub-0.19.0:
      Successfully uninstalled huggingface-hub-0.19.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00


In [2]:

import re
import torch
import transformers
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
from datasets import load_dataset

import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset = load_dataset("glue", "sst2")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [4]:
train = dataset['train']
val = dataset['validation']


In [5]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")


#### Data preprocessing function

In [6]:
def preprocess_text(text):
    # Remove special characters, links, and user mentions
    text = re.sub(r'\b@\w+\b', '', text)  # Remove user mentions
    text = re.sub(r'http\S+', '', text)   # Remove links
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters

    # Tokenize and join the text
    return ' '.join(tokenizer.tokenize(text))


In [7]:
X_train = tokenizer(list(train['sentence']), truncation=True, padding=True, return_tensors='pt', max_length=100, return_attention_mask=True)
X_val = tokenizer(list(val['sentence']), truncation=True, padding=True, return_tensors='pt', max_length=100, return_attention_mask=True)

# Get labels
y_train = torch.tensor(list(train['label']))
y_val = torch.tensor(list(val['label']))


In [8]:
batch_size = 32

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/sentimental_analysis',
    evaluation_strategy="steps",
    save_total_limit=2,
    save_steps=500,
    eval_steps=500,
    report_to="tensorboard",
    per_device_train_batch_size=batch_size,
    num_train_epochs=1,
    learning_rate=2e-5,
    load_best_model_at_end=True,
)


In [9]:
pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.3 MB/s eta 0:00:00


In [10]:
trainer = Trainer(
    model=DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3),  # Three labels: positive, negative, neutral
    args=training_args,
    train_dataset=[{'input_ids': X_train['input_ids'][i], 'attention_mask': X_train['attention_mask'][i], 'labels': y_train[i]} for i in range(len(X_train['input_ids']))],
    eval_dataset=[{'input_ids': X_val['input_ids'][i], 'attention_mask': X_val['attention_mask'][i], 'labels': y_val[i]} for i in range(len(X_val['input_ids']))],
    compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids, p.predictions.argmax(axis=1))}
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
train_results=trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,0.325900,0.240954,0.912844
1000,0.227200,0.244091,0.903670
1500,0.196400,0.280504,0.899083
2000,0.184600,0.254702,0.908257


In [12]:
from sklearn.metrics import classification_report
import torch

# Assuming you've already trained your model and loaded it into the trainer
# You can get the model from the trainer
model = trainer.model

# Put your evaluation dataset into a DataLoader
eval_dataloader = trainer.get_eval_dataloader()

# Set the model in evaluation mode
model.eval()

val_predictions = []
with torch.no_grad():
    for batch in eval_dataloader:
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        labels = batch['labels'].to(model.device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        # Get the predicted labels
        predicted_labels = outputs.logits.argmax(dim=1)
        val_predictions.extend(predicted_labels.tolist())

# Generate the classification report with the correct target names
class_report = classification_report(y_val, val_predictions, target_names=['positive',  'negative'])

# Print the classification report
print(class_report)


              precision    recall  f1-score   support

    positive       0.89      0.93      0.91       428
    negative       0.93      0.89      0.91       444

    accuracy                           0.91       872
   macro avg       0.91      0.91      0.91       872
weighted avg       0.91      0.91      0.91       872



In [13]:
model.save_pretrained("/content/drive/MyDrive/sentimental_analysis")
tokenizer.save_pretrained("/content/drive/MyDrive/sentimental_analysis")

('/content/drive/MyDrive/sentimental_analysis/tokenizer_config.json',
 '/content/drive/MyDrive/sentimental_analysis/special_tokens_map.json',
 '/content/drive/MyDrive/sentimental_analysis/vocab.txt',
 '/content/drive/MyDrive/sentimental_analysis/added_tokens.json')